In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torch.utils.data import sampler
from torchvision import models


import torchvision
from torchvision import datasets, transforms

import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import os
import shutil
from PIL import Image

In [ ]:
# use GPU
USE_GPU = True

dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

In [ ]:
# visualize some training images
data_dir = 'OCT2017/train'
train_dir = 'data/train'
val_dir = 'data/val'
test_dir = 'data/test'
class_names = ['CNV', 'DME', 'DRUSEN', 'NORMAL']

num_images = 5

fig, axes = plt.subplots(len(class_names), num_images, figsize=(num_images * 2, len(class_names) * 2))
fig.suptitle('Sample Images from Each Class', fontsize=16)

for i, class_name in enumerate(class_names):
    class_dir = os.path.join(train_dir, class_name)
    image_files = os.listdir(class_dir)[:num_images]
    for j, image_file in enumerate(image_files):
        image_path = os.path.join(class_dir, image_file)
        image = Image.open(image_path)
        ax = axes[i, j]
        ax.imshow(image, cmap='gray')
        ax.axis('off')
        ax.set_title(class_name)

plt.tight_layout()
plt.show()

In [ ]:
# run only first time to split data
# # Ensure the directories exist
# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(val_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)
#
# # Create train, val, test folders for each class
# for folder in ['CNV', 'DME', 'DRUSEN', 'NORMAL']:
#     os.makedirs(os.path.join(train_dir, folder), exist_ok=True)
#     os.makedirs(os.path.join(val_dir, folder), exist_ok=True)
#     os.makedirs(os.path.join(test_dir, folder), exist_ok=True)
#
# # Split data into train, validation, and test sets
# def split_data(source_dir, train_dir, val_dir, test_dir, split_ratio=(0.8, 0.1, 0.1)):
#     for category in os.listdir(source_dir):
#         category_path = os.path.join(source_dir, category)
#         if os.path.isdir(category_path):
#             files = os.listdir(category_path)
#
#             train_files, test_files = train_test_split(files, test_size=0.1, random_state=42)
#             train_files, val_files = train_test_split(train_files, test_size=0.1 / 0.9, random_state=42)
#
#             for file in train_files:
#                 shutil.move(os.path.join(category_path, file), os.path.join(train_dir, category, file))
#
#             for file in val_files:
#                 shutil.move(os.path.join(category_path, file), os.path.join(val_dir, category, file))
#
#             for file in test_files:
#                 shutil.move(os.path.join(category_path, file), os.path.join(test_dir, category, file))
#
# # Call the function to split the data
# split_data(data_dir, train_dir, val_dir, test_dir)
#
# train_transforms = transforms.Compose([
#     transforms.RandomResizedCrop(224),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     #     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])
#
# val_test_transforms = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     #     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])
#
# batch_size = 32
#
# # Load datasets with ImageFolder
# train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
# val_dataset = datasets.ImageFolder(val_dir, transform=val_test_transforms)
# test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transforms)
#
# # Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
#
# # Get dataset sizes
# train_size = len(train_dataset)
# val_size = len(val_dataset)
# test_size = len(test_dataset)
#
# # Get class names
# class_names = train_dataset.classes
#
# print(f"Classes: {class_names}")
# print(f"Train dataset size: {train_size}")
# print(f"Validation dataset size: {val_size}")
# print(f"Test dataset size: {test_size}")
#
# # Example of using the data loader to get a batch of training data
# example_batch = next(iter(train_loader))
# images, labels = example_batch
# print(f"Batch of images shape: {images.shape}")
# print(f"Batch of labels: {labels}")

In [ ]:
# load data
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    #     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    #     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

batch_size = 32

# Load datasets with ImageFolder
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_test_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Get dataset sizes
train_size = len(train_dataset)
val_size = len(val_dataset)
test_size = len(test_dataset)

# Get class names
class_names = train_dataset.classes

print(f"Classes: {class_names}")
print(f"Train dataset size: {train_size}")
print(f"Validation dataset size: {val_size}")
print(f"Test dataset size: {test_size}")

# Example of using the data loader to get a batch of training data
example_batch = next(iter(train_loader))
images, labels = example_batch
print(f"Batch of images shape: {images.shape}")
print(f"Batch of labels: {labels}")

In [ ]:
# images after preprocessing
index_to_class = {
    0: 'CNV',
    1: 'DME',
    2: 'DRUSSEN',
    3: 'NORMAL'
}

fig, axes = plt.subplots(4, 5, figsize=(5 * 2, 4 * 2))
fig.suptitle('Sample Images after Preprocessing', fontsize=16)

for i in range(4):
    for j in range(5):
        image = images[i * 4 + j]
        image = np.transpose(image, (1, 2, 0))
        ax = axes[i, j]
        ax.imshow(image, cmap='gray')
        ax.axis('off')
        ax.set_title(index_to_class.get(int(labels[i * 4 + j])))

plt.tight_layout()
plt.show()

In [ ]:
# utility funcs
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def check_accuracy(loader, model):
    if loader == train_loader:
        print('Checking accuracy on training set')
    elif loader == val_loader:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')

    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return 100 * acc

def train(model, optimizer, epochs=1):
    model = model.to(device=device)
    acc_max = 0
    for e in range(epochs):
        train_loss, correct = 0, 0
        for t, (x, y) in enumerate(train_loader):

            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)
            train_loss += loss.item()
            correct += (scores.argmax(1) == y).sum()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        average_train_loss = train_loss / len(train_loader)
        accuracy = correct / len(train_loader.dataset)
        print('Epoch %d, loss = %.4f, acc = %.4f' % (e, average_train_loss, accuracy))
        check_accuracy(val_loader, model)
        print()

In [ ]:
# train with print time
import time  # Import the time module

def train(model, optimizer, epochs=1):
    model = model.to(device=device)
    acc_max = 0
    total_time = 0  # Initialize total time for averaging later

    for e in range(epochs):
        train_loss, correct = 0, 0
        start_time = time.time()  # Record the start time of the epoch

        for t, (x, y) in enumerate(train_loader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)
            train_loss += loss.item()
            correct += (scores.argmax(1) == y).sum()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        end_time = time.time()  # Record the end time of the epoch
        epoch_duration = end_time - start_time  # Calculate the duration of the epoch
        total_time += epoch_duration  # Accumulate the total time

        average_train_loss = train_loss / len(train_loader)
        accuracy = correct / len(train_loader.dataset)
        print('Epoch %d, loss = %.4f, acc = %.4f, duration = %.2f seconds' % (e, average_train_loss, accuracy, epoch_duration))
        check_accuracy(val_loader, model)
        print()

    average_epoch_time = total_time / epochs  # Calculate the average time per epoch
    print('Average training time per epoch: %.2f seconds' % average_epoch_time)

# Make sure the device and dtype are defined before calling this function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=channel_1, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=channel_1, out_channels=channel_2, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.fc1 = nn.Linear(channel_2 * 56 * 56, num_classes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = flatten(x)
        x = self.fc1(x)
        return x

In [ ]:
print_every = 500

channel_1 = 64
channel_2 = 32
learning_rate = 1e-3
num_classes = len(train_dataset.classes)

model = SimpleCNN(in_channel=3, channel_1=channel_1, channel_2=channel_2, num_classes=num_classes)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=5)

In [ ]:
cnn = model
check_accuracy(test_loader, cnn)

In [ ]:
# unet
class UNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(UNet, self).__init__()

        self.down1 = self.conv_stage(in_channels, 64)
        self.down2 = self.conv_stage(64, 128)
        self.down3 = self.conv_stage(128, 256)
        self.down4 = self.conv_stage(256, 512)

        self.bottleneck = self.conv_stage(512, 1024)

        self.up1 = self.up_conv_stage(1024, 512)
        self.up2 = self.up_conv_stage(512, 256)
        self.up3 = self.up_conv_stage(256, 128)
        self.up4 = self.up_conv_stage(128, 64)

        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(64, num_classes)

    def conv_stage(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def up_conv_stage(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)

        bn = self.bottleneck(d4)

        u1 = self.up1(bn)
        u2 = self.up2(u1 + d4)  # Skip connection
        u3 = self.up3(u2 + d3)
        u4 = self.up4(u3 + d2)

        pooled = self.global_avg_pool(u4 + d1)  # Final skip connection
        flat = pooled.view(pooled.size(0), -1)
        out = self.fc(flat)

        return out

In [ ]:
print_every = 500

channel_1 = 64
channel_2 = 32
learning_rate = 1e-4
num_classes = len(train_dataset.classes)

model = UNet(in_channels=3, num_classes=num_classes)  # for CNV, DME, DRUSEN, NORMAL

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay = 1e-5)

train(model, optimizer, epochs=30)

In [ ]:
unet = model
check_accuracy(test_loader, unet)

In [ ]:
# unet with resnet pretrained weight
class UNetResNet(nn.Module):
    def __init__(self, num_classes):
        super(UNetResNet, self).__init__()
        resnet = models.resnet34(pretrained=True)
        self.base_layers = list(resnet.children())  # all layers except the last fully connected layer

        # Down path
        self.enc1 = nn.Sequential(*self.base_layers[:3])  # size=(N, 64, x.H/2, x.W/2)
        self.enc2 = nn.Sequential(*self.base_layers[3:5])  # size=(N, 64, x.H/4, x.W/4)
        self.enc3 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)
        self.enc4 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)

        # Bottleneck
        self.bottleneck = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)

        # Up path
        self.up4 = self.up_conv_stage(512, 256)
        self.up3 = self.up_conv_stage(256, 128)
        self.up2 = self.up_conv_stage(128, 64)
        self.up1 = self.up_conv_stage(64, 64)

        # Final output
        self.final_conv = nn.Conv2d(64, num_classes, 1)  # num_classes output channels
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))  # Pool to make spatial dimensions 1x1


    def up_conv_stage(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder part
        enc1 = self.enc1(x)
        enc2 = self.enc2(enc1)
        enc3 = self.enc3(enc2)
        enc4 = self.enc4(enc3)

        # Bottleneck
        bottleneck = self.bottleneck(enc4)

        # Decoder part
        dec4 = self.up4(bottleneck) + enc4
        dec3 = self.up3(dec4) + enc3
        dec2 = self.up2(dec3) + enc2
        dec1 = self.up1(dec2) + enc1

        # Final output
        out = self.final_conv(dec1)
        out = self.adaptive_pool(out)  # Reduce to 1x1 spatially
        out = torch.flatten(out, 1)  # Flatten the outputs into shape [N, num_classes]
        return out

In [ ]:
print_every = 500

channel_1 = 64
channel_2 = 32
learning_rate = 1e-4
model = UNetResNet(num_classes=4)

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

train(model, optimizer, epochs=30)

In [ ]:
unet_pretrained = model
check_accuracy(test_loader, unet_pretrained)

In [ ]:
# efficientnet
class MBConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        super(MBConv, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride
        self.expand_ratio = expand_ratio
        hidden_dim = in_channels * expand_ratio

        self.expansion = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dim, kernel_size=1, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True)
        ) if expand_ratio != 1 else nn.Identity()

        self.depthwise = nn.Sequential(
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3, stride=stride, padding=1, groups=hidden_dim, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True)
        )

        self.projection = nn.Sequential(
            nn.Conv2d(hidden_dim, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

        self.use_residual = self.stride == 1 and in_channels == out_channels

    def forward(self, x):
        identity = x
        x = self.expansion(x)
        x = self.depthwise(x)
        x = self.projection(x)
        if self.use_residual:
            x = x + identity
        return x

In [ ]:
class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes=4):
        super(CustomEfficientNet, self).__init__()
        self.initial_conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace=True)
        )

        self.mb_conv1 = MBConv(32, 16, stride=1, expand_ratio=1)
        self.mb_conv2 = MBConv(16, 24, stride=2, expand_ratio=6)
        self.mb_conv3 = MBConv(24, 40, stride=2, expand_ratio=6)
        self.mb_conv4 = MBConv(40, 80, stride=2, expand_ratio=6)
        self.mb_conv5 = MBConv(80, 112, stride=1, expand_ratio=6)
        self.mb_conv6 = MBConv(112, 192, stride=2, expand_ratio=6)
        self.mb_conv7 = MBConv(192, 320, stride=1, expand_ratio=6)

        self.final_conv = nn.Sequential(
            nn.Conv2d(320, 1280, kernel_size=1, bias=False),
            nn.BatchNorm2d(1280),
            nn.ReLU6(inplace=True)
        )

        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.initial_conv(x)
        x = self.mb_conv1(x)
        x = self.mb_conv2(x)
        x = self.mb_conv3(x)
        x = self.mb_conv4(x)
        x = self.mb_conv5(x)
        x = self.mb_conv6(x)
        x = self.mb_conv7(x)
        x = self.final_conv(x)
        x = self.global_pool(x)
        x = torch.flatten(x, 1)  # Flatten the tensor
        x = self.classifier(x)
        return x

In [ ]:
print_every = 500

channel_1 = 64
channel_2 = 32
learning_rate = 1e-4
num_classes = len(train_dataset.classes)

model = CustomEfficientNet(num_classes=num_classes)

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

train(model, optimizer, epochs=50)

In [ ]:
efficient = model
check_accuracy(test_loader, efficient)

In [ ]:
# densenet
class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(DenseLayer, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = torch.cat([x, out], 1)
        return out

class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, n_layers):
        super(DenseBlock, self).__init__()
        layers = []
        for i in range(n_layers):
            layers.append(DenseLayer(in_channels + i * growth_rate, growth_rate))
        self.dense_block = nn.Sequential(*layers)

    def forward(self, x):
        return self.dense_block(x)

class TransitionLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionLayer, self).__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.pool = nn.AvgPool2d(2)

    def forward(self, x):
        x = self.conv(F.relu(self.bn(x)))
        x = self.pool(x)
        return x

class DenseNet(nn.Module):
    def __init__(self, num_classes=10, growth_rate=12, block_config=(6, 12, 24, 16), init_channels=24):
        super(DenseNet, self).__init__()
        self.growth_rate = growth_rate
        self.init_channels = init_channels

        # Initial convolution
        self.conv1 = nn.Conv2d(3, init_channels, kernel_size=3, padding=1, bias=False)

        # Dense blocks and transition layers
        num_channels = init_channels
        layers = []
        for i, num_layers in enumerate(block_config):
            layers.append(DenseBlock(num_channels, growth_rate, num_layers))
            num_channels += num_layers * growth_rate
            if i != len(block_config) - 1:
                layers.append(TransitionLayer(num_channels, num_channels // 2))
                num_channels = num_channels // 2
        self.features = nn.Sequential(*layers)

        # Classification layer
        self.bn = nn.BatchNorm2d(num_channels)
        self.fc = nn.Linear(num_channels, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.features(x)
        x = F.relu(self.bn(x))
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [ ]:
subset_size = 50000  # the number of images in the subset
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
np.random.shuffle(indices)

subset_indices = indices[:subset_size]
train_subset = Subset(train_dataset, subset_indices)

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
print_every = 200

channel_1 = 64
channel_2 = 32
learning_rate = 1e-3
num_classes = len(train_dataset.classes)

model = DenseNet(num_classes=num_classes, growth_rate=12, block_config=(6, 12, 24, 16), init_channels=24)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

In [ ]:
dense = model
check_accuracy(test_loader, dense)